</br>

# Sampleプログラム：3次元熱水分同時移動解析  

### やること
- 3次元データの入力関係
- 常微分方程式による解法
- 境界条件の入力関係

## 手順0：計算に必要なモジュールを読み込む  

In [3]:
using Pkg
Pkg.add("Plots")

    Updating registry at `C:\Users\takat\.julia\registries\General.toml`
   Resolving package versions...
   Installed libfdk_aac_jll ─────────────── v2.0.3+0
   Installed LoggingExtras ──────────────── v1.1.0
   Installed Libmount_jll ───────────────── v2.40.2+0
   Installed JpegTurbo_jll ──────────────── v3.0.4+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed LERC_jll ───────────────────── v4.0.0+0
   Installed libdecor_jll ───────────────── v0.2.2+0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.6+0
   Installed Unitful ────────────────────── v1.21.1
   Installed ConcurrentUtilities ────────── v2.4.2
   Installed Opus_jll ───────────────────── v1.3.3+0
   Installed Measures ───────────────────── v0.3.2
   Installed RelocatableFolders ─────────── v1.0.1
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed Grisu ──────────────────────── v1.0.2
   Installed RecipesPipeline ────────────── v0.6.12
   Installed PlotUtils ──────────────────── v1.4.3
   Ins

In [4]:
Pkg.add("XLSX")

   Resolving package versions...
   Installed ArgCheck ─────── v2.4.0
   Installed ZipFile ──────── v0.10.1
   Installed XLSX ─────────── v0.10.4
   Installed EzXML ────────── v1.2.0
   Installed ZipArchives ──── v2.4.0
   Installed CodecInflate64 ─ v0.1.3
   Installed InputBuffers ─── v1.0.0
    Updating `C:\Users\takat\.julia\environments\v1.11\Project.toml`
  [fdbf4ff8] + XLSX v0.10.4
    Updating `C:\Users\takat\.julia\environments\v1.11\Manifest.toml`
  [dce04be8] + ArgCheck v2.4.0
  [6309b1aa] + CodecInflate64 v0.1.3
  [8f5d6c58] + EzXML v1.2.0
  [0c81fc1b] + InputBuffers v1.0.0
  [fdbf4ff8] + XLSX v0.10.4
  [49080126] + ZipArchives v2.4.0
  [a5390f91] + ZipFile v0.10.1
Precompiling project...
    872.3 ms  ✓ ArgCheck
    779.8 ms  ✓ InputBuffers
    908.2 ms  ✓ ZipFile
    973.4 ms  ✓ CodecInflate64
   1132.8 ms  ✓ EzXML
   4433.3 ms  ✓ ZipArchives
   1282.9 ms  ✓ XLSX
  7 dependencies successfully precompiled in 7 seconds. 214 already precompiled.


In [ ]:
using Dates # 解析内時間を計算するためのモジュール
using Plots
using XLSX
include("./module/cell.jl"); # 多孔質材料の構成要素を表すモジュール
include("./module/air.jl");  # 空間の構成要素を表すモジュール
include("./module/boundary_condition.jl"); # 境界条件を表すモジュール
include("./module/transfer_in_media.jl"); # 材料中における熱・水分の移動を表すモジュール
include("./module/climate.jl"); # 気象データを取り扱うモジュール
include("./logger.jl"); # 計算結果を保存するモジュール

LoadError: LoadError: UndefVarError: `Climate` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at c:\Users\takat\OneDrive - Kyoto University\ドキュメント\解析プログラム\[Julia]熱水分同時移動標準解析プログラム\[Julia]熱水分同時移動解析標準プログラムver2.2.1\module\boundary_condition.jl:112

## 手順１：計算対象モデルを構築する  

3次元のモデリング方法はその選択肢がいくつかあるが、ここではinput_data内の3D_dataを活用した入力方法について示す。  
なお、壁の基本構造は1次元の解析と同じであるが、Cell構造体に加え、Air構造体、また断熱や断湿などの境界条件を表すBoundaryCondition構造体から構成される配列をtarget_modelとする。

3次元データの入力方法だが、./input_data/3D_data/2D_property_data内の 
- 2D_property_data_番号.csv
というファイルおよび、./input_data/3D_data内の下記のファイルからそれぞれモデリングを行う。
- dx_data.csv
- dy_data.csv
- dz_data.csv
- property_information.csv  

基本的な方針は、2次元平面で描かれるデータを複数つなぎ合わせることで3次元のデータを表現することとなる。  
2D_property_dataは2次元（x軸断面）のデータを表し、CSVファイルの横方向がy軸、縦方向がz軸に相当する。  
このファイルに、壁および境界面のプロパティ番号を記入する。  

ここで記入したプロパティ番号はproperty_information.csvにて定義を行い、  
多孔質材料である場合は材料の名称を、空気である場合はその名称を、境界である場合は境界の名称を記載する。  
また、モデリングデータの寸法情報はそれぞれdx,dy,dz_data内に格納されており、2D_property_dataと合わせた寸法情報を記入すること。  
（空間や境界条件の寸法は便宜上0を入れておくと良い。）

## 1.1 空間の質点の定義

3次元の計算を行うにあたり室空気の定義を行う。  
これは、CV中のCell構造体と異なり、室空気は1質点系として近似することに由来する。  
室空気はAir構造体により設定し、空間情報（体積）、熱力学的状態量、伝達係数の主に3つを有する。  
Air構造体の詳細はair.ipynbのファイルを確認すること。  

なお、室空気は後述のproperty_informationとの対応関係を考慮し辞書型にて定義する。

In [4]:
# 室空気を辞書型として定義
air = Dict()

# キーの値はproperty_information内のnameと対応させる。
air["air_in"]   = Air( name = "air_in", temp = 10.0 + 273.15, rh = 0.6 )
air["air_mid"]  = Air( name = "air_mid",temp = 15.0 + 273.15, rh = 0.6 )
air["air_out"]  = Air( name = "air_out",temp = 20.0 + 273.15, rh = 0.6 )

Air(0, "air_out", 0.0, 0.0, 0.0, 0.0, 293.15, 0.6, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

## 1.2 3次元データ読み込みのための関数

In [ ]:
function read_multi2D_geometry_data(file_number::Int, temp_init::Float64, rh_init::Float64)

    # 3次元のプロパティ情報の読み込み
    file_path = "./input_data/3D_data/2D_property_data/2D_property_data.xlsx"
    xf = XLSX.readxlsx(file_path)
    readed_property_data = [ Matrix(DataFrame(XLSX.readtable(file_path, string(i), header = false))) for i = 1 : length(XLSX.sheetnames(xf)) ]
    #readed_property_data = [ CSV.File("./input_data/3D_data/2D_property_data/"*"2D_property_data_"*string(i)*".csv", header = false) for i = 1 : file_number ]
    property_data = permutedims([ readed_property_data[x][y,z] for x = 1:length(readed_property_data), y = 1:length(readed_property_data[1][:,1]), z = 1:length(readed_property_data[1][1,:]) ], [1,3,2] )
    property_info = DataFrame(CSV.File("./input_data/3D_data/property_information.csv", header = 1))
    property_name = [ filter(row -> row.number == property_data[x,y,z], property_info).name[1] for x = 1:length(readed_property_data), y = 1:length(readed_property_data[1][1,:]), z = 1:length(readed_property_data[1][:,1]) ]

    # 各CVの大きさ（dx, dy, dz）情報の読み込み
    dx_data = CSV.File("./input_data/3D_data/dx_data.csv", header = 1)
    dy_data = CSV.File("./input_data/3D_data/dy_data.csv", header = 1)
    dz_data = CSV.File("./input_data/3D_data/dz_data.csv", header = 1)
    
    # Cellおよび空間を含めた構造体の作成
    target_model = [if property_data[x,y,z] < 20; BC_Neumann() 
                    elseif property_data[x,y,z] < 100;BC_Robin() 
                    else; Cell() end 
                    for x = 1 : length(property_data[:,1,1]), y = 1 : length(property_data[1,:,1]), z = 1 : length(property_data[1,1,:]) ]    

    # 各CVの情報を入力する。
    #for x = 1 : length(dx_data.i) , y = 1 : length(dy_data.i), z = 1 : length(dz_data.i)
    for x = 1 : length(target_model[:,1,1]) , y = 1 : length(target_model[1,:,1]), z = 1 : length(target_model[1,1,:])
        # 第二種境界条件情報の読み取り
        if property_data[x,y,z] < 20

        # 各点における空間情報の読み取り
        elseif property_data[x,y,z] < 100
            target_model[x, y, z].air = air[filter(row -> row.number == property_data[x,y,z], property_info).name[1]]
            target_model[x, y, z].air.dx  = dx_data.dx[x]
            target_model[x, y, z].air.dy  = dy_data.dy[y]
            target_model[x, y, z].air.dz  = dz_data.dz[z]
            target_model[x, y, z].air.vol = target_model[x, y, z].air.vol + dx_data.dx[x] * dy_data.dy[y] * dz_data.dz[z]
            if filter(row -> row.number == property_data[x,y,z], property_info).alphac[1]≠0; target_model[x,y,z].alphac = filter(row -> row.number == property_data[x,y,z], property_info).alphac[1] end
            # 放射熱伝達率が0とすると値が4.4にされてしまうので注意
            if filter(row -> row.number == property_data[x,y,z], property_info).alphar[1]≠0; target_model[x,y,z].alphar = filter(row -> row.number == property_data[x,y,z], property_info).alphar[1] end
            if filter(row -> row.number == property_data[x,y,z], property_info).alpha[1]≠0; target_model[x,y,z].alpha = filter(row -> row.number == property_data[x,y,z], property_info).alpha[1]
            else target_model[x,y,z].alpha = target_model[x,y,z].alphac + target_model[x,y,z].alphar end
            if filter(row -> row.number == property_data[x,y,z], property_info).aldm[1]≠0; target_model[x,y,z].aldm = filter(row -> row.number == property_data[x,y,z], property_info).aldm[1]
            else target_model[x,y,z].aldm = aldm_by_alphac(target_model[x,y,z]) end

        # 各点における壁番号の読み取り
        # なお、i,dx,dx2,temp,miu（水分化学ポテンシャル）,material_nameは入力必須項目である。
        else
            target_model[x, y, z].i   = [ x, y, z ]
            target_model[x, y, z].dx  = dx_data.dx[x]
            target_model[x, y, z].dx2 = dx_data.dx2[x]
            target_model[x, y, z].dy  = dy_data.dy[y]
            target_model[x, y, z].dy2 = dy_data.dy2[y]
            target_model[x, y, z].dz  = dz_data.dz[z]
            target_model[x, y, z].dz2 = dz_data.dz2[z]
            # 位置座標の設定
            if x > 2 && x < length(target_model[:,1,1]) -1
                #println(target_model[x, y, z])
                #target_model[x, y, z].xyz[1] = target_model[x-1, y, z].xyz[1] + target_model[x, y, z].dx - target_model[x, y, z].dx2 
            end
            if y > 2 && x < length(target_model[1,:,1]) -1
                #println(target_model[x, y, z])
                #target_model[x, y, z].xyz[2] = target_model[x, y-1, z].xyz[2] + target_model[x, y, z].dy - target_model[x, y, z].dy2 
            end
            if z > 2 && x < length(target_model[1,1,:]) -1
                #println(target_model[x, y, z])
                #target_model[x, y, z].xyz[3] = target_model[x, y, z-1].xyz[3] + target_model[x, y, z].dz - target_model[x, y, z].dz2 
            end
            target_model[x, y, z].temp= temp_init
            target_model[x, y, z].rh  = rh_init
            target_model[x, y, z].miu = convertRH2Miu( temp = target_model[x, y, z].temp, rh = target_model[x, y, z].rh )
            target_model[x, y, z].material_name = filter(row -> row.number == property_data[x,y,z], property_info).name[1]
        end
    end
    
    return target_model
end 
read_multi2D_geometry_data(;file_number::Int, temp_init::Float64, rh_init::Float64) = read_multi2D_geometry_data(file_number, temp_init, rh_init)

read_multi2D_geometry_data (generic function with 2 methods)

In [6]:
target_model = read_multi2D_geometry_data(file_number = 7, temp_init = 15.0 + 273.15, rh_init = 0.6)

LoadError: UndefVarError: `XLSX` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
target_model[:, :, 1]

LoadError: UndefVarError: `target_model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## 手順２：流量計算式および収支式を構築する

In [5]:
function cal3D_new_temp_miu( target_model, dt )
    ###################################
    #　初期設定    
    ntemp = [ 0.0 for x = 2 : length(target_model[:,1,1])-1, y = 2 : length(target_model[1,:,1])-1, z = 2 : length(target_model[1,1,:])-1 ]
    nmiu  = [ 0.0 for x = 2 : length(target_model[:,1,1])-1, y = 2 : length(target_model[1,:,1])-1, z = 2 : length(target_model[1,1,:])-1 ]
    for x = 1 : length(target_model[:,1,1]), y = 1 : length(target_model[1,:,1]) - 1, z = 1 : length(target_model[1,1,:])
        if typeof(target_model[x,y,z]) == BC_Robin
            target_model[x,y,z].air.H_wall = 0.0
            target_model[x,y,z].air.J_wall = 0.0
        end
    end

    # 変数の設定
    qx  = ones(length(target_model[:,1,1])-1, length(target_model[1,:,1]),   length(target_model[1,1,:])   )
    qy  = ones(length(target_model[:,1,1]),   length(target_model[1,:,1])-1, length(target_model[1,1,:])   )
    qz  = ones(length(target_model[:,1,1]),   length(target_model[1,:,1]),   length(target_model[1,1,:])-1 )
    jvx = ones(length(target_model[:,1,1])-1, length(target_model[1,:,1]),   length(target_model[1,1,:])   )
    jvy = ones(length(target_model[:,1,1]),   length(target_model[1,:,1])-1, length(target_model[1,1,:])   )
    jvz = ones(length(target_model[:,1,1]),   length(target_model[1,:,1]),   length(target_model[1,1,:])-1 )
    jlx = ones(length(target_model[:,1,1])-1, length(target_model[1,:,1]),   length(target_model[1,1,:])   )
    jly = ones(length(target_model[:,1,1]),   length(target_model[1,:,1])-1, length(target_model[1,1,:])   )
    jlz = ones(length(target_model[:,1,1]),   length(target_model[1,:,1]),   length(target_model[1,1,:])-1 )

    ###################################
    # 壁体内部の流量計算
    # x軸方向
    for x = 1 : length(target_model[:,1,1]) - 1, y = 1 : length(target_model[1,:,1]), z = 1 : length(target_model[1,1,:])
        qx[x,y,z]  =  cal_qx( target_model[x,y,z], target_model[x+1,y,z] )
        jvx[x,y,z] = cal_jvx( target_model[x,y,z], target_model[x+1,y,z] )
        jlx[x,y,z] = cal_jlx( target_model[x,y,z], target_model[x+1,y,z], 0.0 )
        # 空気層に流入・流出する熱流量・水分流量の計算
        if typeof(target_model[x,y,z]) == BC_Robin && typeof(target_model[x+1,y,z]) == Cell
            target_model[x,y,z].air.H_wall = target_model[x,y,z].air.H_wall - dy(target_model[x+1,y,z])*dz(target_model[x+1,y,z])*qx[x,y,z]
            target_model[x,y,z].air.J_wall = target_model[x,y,z].air.J_wall - dy(target_model[x+1,y,z])*dz(target_model[x+1,y,z])*jvx[x,y,z]
        elseif typeof(target_model[x+1,y,z]) == BC_Robin && typeof(target_model[x,y,z]) == Cell
            target_model[x+1,y,z].air.H_wall = target_model[x+1,y,z].air.H_wall + dy(target_model[x,y,z])*dz(target_model[x,y,z])*qx[x,y,z]
            target_model[x+1,y,z].air.J_wall = target_model[x+1,y,z].air.J_wall + dy(target_model[x,y,z])*dz(target_model[x,y,z])*jvx[x,y,z]
        end
    end

    # y軸方向
    for x = 1 : length(target_model[:,1,1]), y = 1 : length(target_model[1,:,1]) - 1, z = 1 : length(target_model[1,1,:])
        qy[x,y,z]  =  cal_qy( target_model[x,y,z], target_model[x,y+1,z] )
        jvy[x,y,z] = cal_jvy( target_model[x,y,z], target_model[x,y+1,z] )
        jly[x,y,z] = cal_jly( target_model[x,y,z], target_model[x,y+1,z], 0.0 )
        # 空気層に流入・流出する熱流量・水分流量の計算
        if typeof(target_model[x,y,z]) == BC_Robin && typeof(target_model[x,y+1,z]) == Cell
            target_model[x,y,z].air.H_wall = target_model[x,y,z].air.H_wall - dx(target_model[x,y+1,z])*dz(target_model[x,y+1,z])*qy[x,y,z]
            target_model[x,y,z].air.J_wall = target_model[x,y,z].air.J_wall - dx(target_model[x,y+1,z])*dz(target_model[x,y+1,z])*jvy[x,y,z]
        elseif typeof(target_model[x,y+1,z]) == BC_Robin && typeof(target_model[x,y,z]) == Cell
            target_model[x,y+1,z].air.H_wall = target_model[x,y+1,z].air.H_wall + dx(target_model[x,y,z])*dz(target_model[x,y,z])*qy[x,y,z]
            target_model[x,y+1,z].air.J_wall = target_model[x,y+1,z].air.J_wall + dx(target_model[x,y,z])*dz(target_model[x,y,z])*jvy[x,y,z]
        end
    end
    
    # z軸方向
    for x = 1 : length(target_model[:,1,1]), y = 1 : length(target_model[1,:,1]), z = 1 : length(target_model[1,1,:]) - 1
        qz[x,y,z]  =  cal_qz( target_model[x,y,z], target_model[x,y,z+1] )
        jvz[x,y,z] = cal_jvz( target_model[x,y,z], target_model[x,y,z+1] )
        jlz[x,y,z] = cal_jlz( target_model[x,y,z], target_model[x,y,z+1], 1.0 )
        # 空気層に流入・流出する熱流量・水分流量の計算
        if typeof(target_model[x,y,z]) == BC_Robin && typeof(target_model[x,y,z+1]) == Cell
            target_model[x,y,z].air.H_wall = target_model[x,y,z].air.H_wall - dx(target_model[x,y,z+1])*dy(target_model[x,y,z+1])*qz[x,y,z]
            target_model[x,y,z].air.J_wall = target_model[x,y,z].air.J_wall - dx(target_model[x,y,z+1])*dy(target_model[x,y,z+1])*jvz[x,y,z]
        elseif typeof(target_model[x,y,z+1]) == BC_Robin && typeof(target_model[x,y,z]) == Cell
            target_model[x,y,z+1].air.H_wall = target_model[x,y,z+1].air.H_wall + dx(target_model[x,y,z])*dy(target_model[x,y,z])*qz[x,y,z]
            target_model[x,y,z+1].air.J_wall = target_model[x,y,z+1].air.J_wall + dx(target_model[x,y,z])*dy(target_model[x,y,z])*jvz[x,y,z]
        end
    end

    ###################################
    # 熱・水分の収支計算
    for x = 2 : length(target_model[:,1,1])-1, y = 2 : length(target_model[1,:,1])-1, z = 2 : length(target_model[1,1,:])-1
        if typeof(target_model[x,y,z]) == Cell
            ntemp[x-1,y-1,z-1] = cal_newtemp( cell = target_model[x,y,z], 
                dqx = ( qx[x-1,y,z] - qx[x,y,z] ), dqy = ( qy[x,y-1,z] - qy[x,y,z] ), dqz = ( qz[x,y,z-1] - qz[x,y,z] ) , 
                djvx  = - (jvx[x-1,y,z] -jvx[x,y,z] ), djvy = - (jvy[x,y-1,z] -jvy[x,y,z] ), djvz = - (jvz[x,y,z-1] -jvz[x,y,z] ) , time = dt )
            nmiu[x-1,y-1,z-1]  = cal_newmiu(  cell = target_model[x,y,z], 
                djwx = (jvx[x-1,y,z] -jvx[x,y,z] ) + (jlx[x-1,y,z] -jlx[x,y,z] ),
                djwy = (jvy[x,y-1,z] -jvy[x,y,z] ) + (jly[x,y-1,z] -jly[x,y,z] ),
                djwz = (jvz[x,y,z-1] -jvz[x,y,z] ) + (jlz[x,y,z-1] -jlz[x,y,z] ), time = dt ) 
        elseif typeof(target_model[x,y,z]) == BC_Robin
            ntemp[x-1,y-1,z-1] = cal_newtemp(target_model[x,y,z].air, dt)
            nmiu[x-1,y-1,z-1]  = convertRH2Miu( temp=temp(target_model[x,y,z].air), rh =cal_newRH(target_model[x,y,z].air, dt) )
        end
    end
        
    return ntemp, nmiu
end

cal3D_new_temp_miu (generic function with 1 method)

    # 熱・水分の収支計算
    ntemp = [ cal_newtemp( cell = target_model[x,y,z], 
        dqx = ( qx[x-1,y,z] - qx[x,y,z] ), dqy = ( qy[x,y-1,z] - qy[x,y,z] ), dqz = ( qz[x,y,z-1] - qz[x,y,z] ) , 
        djvx  = (jvx[x-1,y,z] -jvx[x,y,z] ), djvy = (jvy[x,y-1,z] -jvy[x,y,z] ), djvz = (jvz[x,y,z-1] -jvz[x,y,z] ) , time = dt ) 
        for x = 2 : length(target_model[:,1,1])-1, y = 2 : length(target_model[1,:,1])-1, z = 2 : length(target_model[1,1,:])-1 ]
    nmiu  = [ cal_newmiu(  cell = target_model[x,y,z], 
        djwx = (jvx[x-1,y,z] -jvx[x,y,z] ) + (jlx[x-1,y,z] -jlx[x,y,z] ),
        djwy = (jvy[x,y-1,z] -jvy[x,y,z] ) + (jly[x,y-1,z] -jly[x,y,z] ),
        djwz = (jvz[x,y,z-1] -jvz[x,y,z] ) + (jlz[x,y,z-1] -jlz[x,y,z] ), time = dt ) 
        for x = 2 : length(target_model[:,1,1])-1, y = 2 : length(target_model[1,:,1])-1, z = 2 : length(target_model[1,1,:])-1 ]

## 手順３：計算時間に関する設定  

手順３～５のうちいくつかは必要に応じて設定する項目である。  


#### 初期時刻の設定  
計算回数を設定するため初期時刻を以下のように設定する。  
※2023/06/17修正

In [6]:
date = DateTime(2004,8,1,0,0,0)

2004-08-01T00:00:00

#### 終了時刻の設定  
同様に終了時刻も設定しておく。

In [ ]:
#end_date = DateTime(2004,8,4,00,00,00)
end_date = DateTime(2004,8,1,00,10,00)

2004-08-02T00:00:00

## 手順４：気象条件に関する設定  
気象条件に関する諸設定として、
1. 気象データの読み込み
2. 気象データを解析内時間に合わせて線形内挿する
といった動作が必要となる。  
特に2.については計算が複雑なため、climate_data.ipynbにその計算方法について示してある。  

なお、読み込んだ気象データはJuliaの標準パッケージであるDataFramesの形式となる。  
読み込み方は、関数input_climate_data()を用い、input_data内にある気象データ（CSV形式）のディレクトリを指定する。  
なお、juliaはヘッダー名称が必須のため、必ず1行目にヘッダー名を記載すること。  
(以下の例では「input_data」フォルダ内にある気象データを読み込むことを想定する。)  
※1 読み込めるのはcsvファイルだけであるためExcelファイルは読み込めない。  
※2 入力される温度・相対湿度は「℃」セルシウス度、「％」相対湿度を想定している。  
温度をケルビンあるいは相対湿度を無次元「－」で利用する場合は要注意

In [8]:
climate_data_in  = input_climate_data(
    file_name = "./input_data/climate_data/climate_data_nijo_otsune.csv", 
    header = 3 );

climate_data_in.air = air["air_in"]

# ロギングインターバルの確認
climate_data_in.logging_interval

temp is not same
rh is not same


10

### 外気側環境条件

In [9]:
climate_data_out = input_climate_data( 
    file_name = "./input_data/climate_data/climate_data_nijo_otsune.csv", 
    header = 3);

climate_data_out.air = air["air_out"]

# ロギングインターバルの確認
climate_data_in.logging_interval

temp is not same
rh is not same


10

## 手順５：計算結果の保存

まずはじめに計算開始前に計算結果を出力するファイルをオープンする。  
ファイルのオープンはopen関数を用いる。  

opne(file名, mode)  
なおmodeには以下の種類がある。

- r：読込みモード
- w：書込みモード(ファイルが存在する場合は上書き)
- a：追記モード
- r+：読込みと書込みどちらも可能なモード(ファイルが存在しない場合はエラー)
- w+：読込みと書込みどちらも可能なモード(ファイルが存在しない場合は新規作成)
- +：読込みと追記のどちらも可能なモード

※モード指定しない場合、デフォルトでは"r"(読込みモード)となる。  

## 5.1 計算結果の出力先ファイルのオープン

In [10]:
#logger_temp = set_logger( "result_sample_temp", 10.0, air_in, wall, air_out )
#logger_rh   = set_logger( "result_sample_rh", 10.0, air_in, wall, air_out )
#logger_pv   = set_logger( "result_sample_pv", 10.0, air_in, wall, air_out )

## 5.2 ヘッダーの書き込み

In [11]:
#write_header_to_logger( logger_temp )
#write_header_to_logger( logger_rh   )
#write_header_to_logger( logger_pv   )

## 5.3 値（初期値）の書き込み

In [12]:
#write_temp_to_logger( logger_temp, Dates.format(date, "yyyy/mm/dd HH:MM") )
#write_RH_to_logger(   logger_rh, Dates.format(date, "yyyy/mm/dd HH:MM") )
#write_Pv_to_logger(   logger_pv, Dates.format(date, "yyyy/mm/dd HH:MM") )

## 手順６：繰り返し（ループ）計算  

ループ計算は手順３で設定した終了時刻まで計算を行うようなプログラムである。  
なお、計算開始から終了まで時間がかかる時はデータを全てロギングするとデータ量が膨大となるため、ある程度期間を指定したほうが良い。  
（例えば下記の「#５：データのロギング」にて、if date.year == end_data.year のように条件づけるなど）

#### 計算刻みdtの設定  
数値解析における微小時間変化dtを以下にて設定する。

In [13]:
#dt = 0.002
dt = 1.0
total_time = 0.0

0.0

In [ ]:
# 計算回数の設定
while date ≠ end_date
    
    # １：環境データの入力
    reset_climate_data( climate_data_in )
    reset_climate_data( climate_data_out )
    
    # ２：新値の計算
    ntemp, nmiu = cal3D_new_temp_miu( target_model, dt )
    
    # ３：値の上書き
    for x = 2 : length(target_model[:,1,1])-1, y = 2 : length(target_model[1,:,1])-1, z = 2 : length(target_model[1,1,:])-1        
        if typeof(target_model[x,y,z]) == Cell
            target_model[x,y,z].temp = ntemp[x-1,y-1,z-1]
            target_model[x,y,z].miu  = nmiu[x-1,y-1,z-1]
        elseif typeof(target_model[x,y,z]) == BC_Robin
            target_model[x,y,z].air.temp = ntemp[x-1,y-1,z-1]
            target_model[x,y,z].air.rh  = convertMiu2RH( temp = ntemp[x-1,y-1,z-1], miu = nmiu[x-1,y-1,z-1] )
        end
    end
    
    # 計算時刻の表示
    # 日本語は避けた方が無難
    #if hour(date) == 0 && minute(date) == 0 && second(date) == 0 && millisecond(date) == 0;
    if mod(minute(date),10) == 0 && second(date) == 0 && millisecond(date) == 0;
        println(Dates.format(date, "yyyy/mm/dd HH:MM"), " 外気：温度" *  string( round(air["air_out"].temp -273.15, digits = 1 ) ) * "[℃]" * " 湿度" * string( round( air["air_out"].rh, digits = 2 ) ) * "[-]" )
    end
        
    # ４：時間経過の計算および時間刻みの更新
    #date = date + Second(dt)
    date = date + Millisecond(dt*1000)
    total_time = total_time + dt
    climate_data_in.date    = date
    climate_data_out.date   = date
    #if total_time > 30.0; dt = 0.005
    #elseif total_time > 180.0; dt = 0.01
    #end
    # 1秒以下の場合、Millisecondを使う。

    # ５：データのロギング  
    if mod(minute(date),10) == 0 && second(date) == 0 && millisecond(date) == 0
        # ファイルの作成
        data_temp = open("./output_data/temp_data_"*string(month(date))*"_"*string(day(date))*"_"*string(hour(date))*"_"*string(minute(date))*".csv", "w")
        data_rh = open("./output_data/rh_data_"*string(month(date))*"_"*string(day(date))*"_"*string(hour(date))*"_"*string(minute(date))*".csv", "w")
        # データの書き込み
        for x = 2 : length(target_model[:,1,1])-1
            println(data_temp, "x=", x )
            println(data_rh,   "x=", x )
            for y = 2 : length(target_model[1,:,1])-1
                for z = 2 : length(target_model[1,1,:])-1        
                    println(data_temp, x, ",", y, ",", z, ",", temp(target_model[x,y,z]) - 273.15 )
                    println(data_rh,   x, ",", y, ",", z, ",", rh(target_model[x,y,z]) )
                end
            end
        end
        close(data_temp)
        close(data_rh)
    end
    
end

2004/08/01 00:00 外気：温度28.8[℃] 湿度0.72[-]
2004/08/01 00:10 外気：温度29.1[℃] 湿度0.59[-]
2004/08/01 00:20 外気：温度29.1[℃] 湿度0.59[-]


In [ ]:
target_model[2,2:end-1,2:end-1]

In [ ]:
#データ作成
Y = 2 : length(target_model[1,:,1])-1
Z = 2 : length(target_model[1,1,:])-1
value = permutedims([temp(target_model[4,y,z]) for y in Y, z in Z ], [2,1])
#各プロット
p1=heatmap(value,title="heatmap")                            #ヒートマップ heatmap(2次元データ,...)
#p2=contour(value,Y,Z,title="contour")                        #コンター図   contour(xデータ,y,z,...)
#p3=wireframe(value,Y,Z,title="wireframe")                    #3Dのワイヤーフレーム表記
#p4=surface(value,Y,Z,title="surface")                        #3Dの曲面表示
#plot(p1,p2,p3,p4,layout=(1,4),size=(1500,250),fmt=:png)  #1×4表記 (少し重たくなるので、後述のフォーマットpng)


In [ ]:
[ temp(target_model[2,y,z]) for y = 2:9, z = 2:17 ]

In [ ]:
[ target_model[2,y,z].miu for y = 2:9, z = 2:17 ]

In [ ]:
date